In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/novel-corona-virus-2019-dataset/time_series_2019_ncov_recovered.csv
/kaggle/input/novel-corona-virus-2019-dataset/2019_nCoV_data.csv
/kaggle/input/novel-corona-virus-2019-dataset/time_series_2019_ncov_confirmed.csv
/kaggle/input/novel-corona-virus-2019-dataset/time_series_2019_ncov_deaths.csv


In [2]:
#Importing Necessary Libraries
import matplotlib.pyplot as plt
import plotly.graph_objects as go 
import seaborn as sns
import plotly
import plotly.express as px
from fbprophet.plot import plot_plotly
from fbprophet import Prophet

In [3]:
#Setting up plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot, plot_mpl
import plotly.offline as py
init_notebook_mode(connected=True)

In [4]:
# Reading dataset
dataset = pd.read_csv('../input/novel-corona-virus-2019-dataset/2019_nCoV_data.csv')

In [5]:
dataset.head(10)

,Sno,Date,Province/State,Country,Last Update,Confirmed,Deaths,Recovered
0,1,01/22/2020 12:00:00,Anhui,China,01/22/2020 12:00:00,1.0,0.0,0.0
1,2,01/22/2020 12:00:00,Beijing,China,01/22/2020 12:00:00,14.0,0.0,0.0
2,3,01/22/2020 12:00:00,Chongqing,China,01/22/2020 12:00:00,6.0,0.0,0.0
3,4,01/22/2020 12:00:00,Fujian,China,01/22/2020 12:00:00,1.0,0.0,0.0
4,5,01/22/2020 12:00:00,Gansu,China,01/22/2020 12:00:00,0.0,0.0,0.0
5,6,01/22/2020 12:00:00,Guangdong,China,01/22/2020 12:00:00,26.0,0.0,0.0
6,7,01/22/2020 12:00:00,Guangxi,China,01/22/2020 12:00:00,2.0,0.0,0.0
7,8,01/22/2020 12:00:00,Guizhou,China,01/22/2020 12:00:00,1.0,0.0,0.0
8,9,01/22/2020 12:00:00,Hainan,China,01/22/2020 12:00:00,4.0,0.0,0.0
9,10,01/22/2020 12:00:00,Hebei,China,01/22/2020 12:00:00,1.0,0.0,0.0


# Data Exploration and Visualisations

# Data Exploration

In [6]:
# Checking if there is any null values
dataset.isnull().any()

Sno               False
Date              False
Province/State     True
Country           False
Last Update       False
Confirmed         False
Deaths            False
Recovered         False
dtype: bool

In [7]:
# Seeing the datset with null rows
dataset[dataset.isnull().any(axis=1)]

,Sno,Date,Province/State,Country,Last Update,Confirmed,Deaths,Recovered
35,36,01/22/2020 12:00:00,NaN,Japan,01/22/2020 12:00:00,2.0,0.0,0.0
36,37,01/22/2020 12:00:00,NaN,Thailand,01/22/2020 12:00:00,2.0,0.0,0.0
37,38,01/22/2020 12:00:00,NaN,South Korea,01/22/2020 12:00:00,1.0,0.0,0.0
73,74,01/23/2020 12:00:00,NaN,Japan,01/23/2020 12:00:00,1.0,0.0,0.0
74,75,01/23/2020 12:00:00,NaN,Thailand,01/23/2020 12:00:00,3.0,0.0,0.0
...,...,...,...,...,...,...,...,...
1706,1707,02/17/2020 22:00:00,NaN,Egypt,2020-02-14T23:53:02,1.0,0.0,0.0
1707,1708,02/17/2020 22:00:00,NaN,Finland,2020-02-12T00:03:12,1.0,0.0,1.0
1709,1710,02/17/2020 22:00:00,NaN,Nepal,2020-02-12T14:43:03,1.0,0.0,1.0
1710,1711,02/17/2020 22:00:00,NaN,Sri Lanka,2020-02-08T03:43:03,1.0,0.0,1.0


In [8]:
# Affected Countries
print(f"Affected Countries are : {dataset['Country'].unique()}")
print(f"Total Affected Countries are : {len(dataset['Country'].unique())}")

Affected Countries are : ['China' 'US' 'Japan' 'Thailand' 'South Korea' 'Mainland China'
 'Hong Kong' 'Macau' 'Taiwan' 'Singapore' 'Philippines' 'Malaysia'
 'Vietnam' 'Australia' 'Mexico' 'Brazil' 'France' 'Nepal' 'Canada'
 'Cambodia' 'Sri Lanka' 'Ivory Coast' 'Germany' 'Finland'
 'United Arab Emirates' 'India' 'Italy' 'Sweden' 'Russia' 'Spain' 'UK'
 'Belgium' 'Others' 'Egypt']
Total Affected Countries are : 34


In [9]:
# Affected States/Provinces
print(f"Affected State/Provinces are : {dataset['Province/State'].unique()}")
print(f"Total Affected State/Provinces are : {len(dataset['Province/State'].unique())}")

Affected State/Provinces are : ['Anhui' 'Beijing' 'Chongqing' 'Fujian' 'Gansu' 'Guangdong' 'Guangxi'
 'Guizhou' 'Hainan' 'Hebei' 'Heilongjiang' 'Henan' 'Hong Kong' 'Hubei'
 'Hunan' 'Inner Mongolia' 'Jiangsu' 'Jiangxi' 'Jilin' 'Liaoning' 'Macau'
 'Ningxia' 'Qinghai' 'Shaanxi' 'Shandong' 'Shanghai' 'Shanxi' 'Sichuan'
 'Taiwan' 'Tianjin' 'Tibet' 'Washington' 'Xinjiang' 'Yunnan' 'Zhejiang'
 nan 'Chicago' 'Illinois' 'California' 'Arizona' 'Ontario'
 'New South Wales' 'Victoria' 'Bavaria' 'British Columbia' 'Queensland'
 'Chicago, IL' 'South Australia' 'Boston, MA' 'Los Angeles, CA'
 'Orange, CA' 'Santa Clara, CA' 'Seattle, WA' 'Tempe, AZ' 'Toronto, ON'
 'San Benito, CA' 'London, ON' 'Madison, WI' 'Cruise Ship'
 'Diamond Princess cruise ship' 'San Diego County, CA' 'San Antonio, TX']
Total Affected State/Provinces are : 62


# Dataset Visualisations

# Bar Plots

## Confirmed Visualisations

Different Color shows different Country

In [10]:
fig = px.bar(dataset, x='Date', y='Confirmed', hover_data=['Province/State', 'Deaths', 'Recovered'], color='Country')
annotations = []
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text='Confirmed bar plot for each country',
                              font=dict(family='Arial',
                                        size=30,
                                        color='rgb(37,37,37)'),
                              showarrow=False))
fig.update_layout(annotations=annotations)
fig.show()

Here taking Mainland Chine as an example and different color shows different State/Provinces 

In [11]:
fig = px.bar(dataset.loc[dataset['Country'] == 'Mainland China'], x='Date', y='Confirmed', hover_data=['Province/State', 'Deaths', 'Recovered'], color='Province/State')
annotations = []
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text='Confirmed bar plot for Mainland China',
                              font=dict(family='Arial',
                                        size=30,
                                        color='rgb(37,37,37)'),
                              showarrow=False))
fig.update_layout(annotations=annotations)
fig.show()

In [12]:
# Saving the image 
py.plot(fig, filename='Confirmed_plot.html')

'Confirmed_plot.html'

## Death Visualisations

In [13]:
fig = px.bar(dataset, x='Date', y='Deaths', hover_data=['Province/State', 'Confirmed', 'Recovered'], color='Country')
annotations = []
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text='Death bar plot for each country',
                              font=dict(family='Arial',
                                        size=30,
                                        color='rgb(37,37,37)'),
                              showarrow=False))
fig.update_layout(annotations=annotations)
fig.show()

In [14]:
fig = px.bar(dataset.loc[dataset['Country'] == 'Mainland China'], x='Date', y='Deaths', hover_data=['Province/State', 'Confirmed', 'Recovered'], color='Province/State')
annotations = []
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text='Death bar plot for Mainland China',
                              font=dict(family='Arial',
                                        size=30,
                                        color='rgb(37,37,37)'),
                              showarrow=False))
fig.update_layout(annotations=annotations)
fig.show()

# Line Plots

## Confirmed PLots

In [15]:
fig = px.line(dataset, x="Date", y="Confirmed", color='Country', hover_data=['Province/State', 'Deaths'])
annotations = []
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text='Confirmed Plot for each Country',
                              font=dict(family='Arial',
                                        size=30,
                                        color='rgb(37,37,37)'),
                              showarrow=False))
fig.update_layout(annotations=annotations)
fig.show()

## Deaths Plots

In [16]:
fig = px.line(dataset, x="Date", y="Deaths", color='Country', hover_data=['Province/State', 'Deaths'])
annotations = []
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text='Death plot for each country',
                              font=dict(family='Arial',
                                        size=30,
                                        color='rgb(37,37,37)'),
                              showarrow=False))
fig.update_layout(annotations=annotations)
fig.show()

# Showing Death's of total countries 

Grouping Dataset by Date

In [17]:
fig = px.line(pd.DataFrame(dataset.groupby('Date')['Confirmed'].sum().reset_index()), x="Date", y="Confirmed")
annotations = []
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text='Showing Deaths of total country',
                              font=dict(family='Arial',
                                        size=30,
                                        color='rgb(37,37,37)'),
                              showarrow=False))
fig.update_layout(annotations=annotations)
fig.show()

The cases are increasing at very high rate !

# Maps Visualisations

A special thansk for **Ranjith KS** whose [kernel](https://www.kaggle.com/ranjithks/ran-coronavirus-exploratory-data-analysis-eda#Exploratory-Data-Analysis) helped me a lot to complete this map visualisation part, most of the code here is of her

In [18]:
pxdf = px.data.gapminder()
country_isoAlpha = pxdf[['country', 'iso_alpha']].drop_duplicates()
country_isoAlpha.rename(columns = {'country':'Country'}, inplace=True)
country_isoAlpha.set_index('Country', inplace=True)
country_map = country_isoAlpha.to_dict('index')

In [19]:
def getCountryIsoAlpha(country):
    try:
        return country_map[country]['iso_alpha']
    except:
        return country

In [20]:
dataset.replace({'Country': 'Mainland China'}, 'China', inplace=True)
dataset['iso_alpha'] = dataset['Country'].apply(getCountryIsoAlpha)

## Confirmed Cases Maps

In [21]:
df_plot = dataset.groupby('iso_alpha').max().reset_index()
fig = px.choropleth(df_plot, locations="iso_alpha",
                    color="Confirmed",
                    hover_data=["Confirmed", "Deaths", "Recovered"],
                    color_continuous_scale="Viridis")
fig.update_geos(fitbounds="locations", visible=True)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, title_text = 'Deaths Cases in World')

fig.show()

In [22]:
fig = px.scatter_geo(dataset, locations="iso_alpha", 
                     color="Confirmed", size='Confirmed', hover_name="Country", 
                    hover_data=["Confirmed", "Deaths", "Recovered"],
                     projection="natural earth", animation_frame="Date")
fig.show()

## Deaths Cases Maps

In [23]:

fig = px.choropleth(df_plot, locations="iso_alpha",
                    color="Deaths",
                    hover_data=["Confirmed", "Deaths", "Recovered"],
                    color_continuous_scale="Viridis")
fig.update_geos(fitbounds="locations", visible=True)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, title_text = 'Deaths Cases in World')

fig.show()

In [24]:
fig = px.scatter_geo(dataset, locations="iso_alpha", 
                     color="Deaths", size='Deaths', hover_name="Country",
                    hover_data=["Confirmed", "Deaths", "Recovered"] ,
                     projection="natural earth", animation_frame="Date")
fig.show()

# Here's come Machine Learning
Prediting Confirmed and Death cases using **Prophet Model**

## Forcasting Confirmed Cases

In [25]:
confirmed_training_dataset = pd.DataFrame(dataset.groupby('Date')['Confirmed'].sum().reset_index()).rename(columns={'Date': 'ds', 'Confirmed': 'y'})
confirmed_training_dataset

,ds,y
0,01/22/2020 12:00:00,555.0
1,01/23/2020 12:00:00,653.0
2,01/24/2020 12:00:00,941.0
3,01/25/2020 22:00:00,2019.0
4,01/26/2020 23:00:00,2794.0
5,01/27/2020 20:30:00,4473.0
6,01/28/2020 23:00:00,6057.0
7,01/29/2020 21:00:00,7783.0
8,01/30/2020 21:30:00,9776.0
9,01/31/2020 19:00:00,11374.0


In [26]:
# Making the Model
prophet = Prophet()
prophet.fit(confirmed_training_dataset)
future = prophet.make_future_dataframe(periods=7)
confirmed_forecast = prophet.predict(future)

# Confirmed Cases Forcasting Visualisation

In [27]:
fig = plot_plotly(prophet, confirmed_forecast)  
annotations = []
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text='Predictions for Confirmed',
                              font=dict(family='Arial',
                                        size=30,
                                        color='rgb(37,37,37)'),
                              showarrow=False))
fig.update_layout(annotations=annotations)
fig

# Forcasing Death Cases

In [28]:
death_training_dataset = pd.DataFrame(dataset.groupby('Date')['Deaths'].sum().reset_index()).rename(columns={'Date': 'ds', 'Deaths': 'y'})
death_training_dataset

,ds,y
0,01/22/2020 12:00:00,0.0
1,01/23/2020 12:00:00,18.0
2,01/24/2020 12:00:00,26.0
3,01/25/2020 22:00:00,56.0
4,01/26/2020 23:00:00,80.0
5,01/27/2020 20:30:00,107.0
6,01/28/2020 23:00:00,132.0
7,01/29/2020 21:00:00,170.0
8,01/30/2020 21:30:00,213.0
9,01/31/2020 19:00:00,259.0


In [29]:
# Making the Model
prophet = Prophet()
prophet.fit(death_training_dataset)
future = prophet.make_future_dataframe(periods=7)
deaths_forecast = prophet.predict(future)

In [30]:
fig = plot_plotly(prophet, deaths_forecast)  
annotations = []
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text='Predictions of Deaths',
                              font=dict(family='Arial',
                                        size=30,
                                        color='rgb(37,37,37)'),
                              showarrow=False))
fig.update_layout(annotations=annotations)
fig

In [31]:
# Saving Death Forcasting  
py.plot(fig, filename='death_forcasting.html')

'death_forcasting.html'

# Accuracy in the  Prophet Model. 

In [32]:
dates=['Feb 16', 'Feb 17']

fig = go.Figure(data=[
    go.Bar(name='Predicted', x=dates, y=[1806, 1921]),
    go.Bar(name='Actial', x=dates, y=[1770, 1868])
])
# Change the bar mode
fig.update_layout(barmode='group', title_text='Predictions and Actual Data from Feb 16 and 17')
fig.show()